# Import Libraries

In [ ]:
from heapdict import heapdict
import os
import json

import pandas as pd

# Load Data

In [ ]:
institutions_file_path = "./result/institutions.json"
with open(institutions_file_path, "r") as file: institutions = json.load(file)

In [ ]:
registered_institutions_file_path = "./input/distribution/registered.txt"

In [ ]:
marksheet_df = pd.read_csv("./result/marksheet.csv")

In [ ]:
options_file_path = "./input/distribution/options.json"
with open(options_file_path, "r") as file: options_map = json.load(file)

# Distribute Slots

In [ ]:
def get_registered_institution_names(institutions, file_path):
    registered_names_set = set()

    with open(file_path, "r") as txt_file:
        target_names = set(line.strip().upper() for line in txt_file if line.strip())

    for institution in institutions:
        all_names = [institution["name"]] + institution["alt_names"]
        if any(n.upper() in target_names for n in all_names):
            registered_names_set.add(institution["name"])
            
            for n in all_names:
                target_names.discard(n.upper())

    if target_names:
        raise ValueError(f"Unrecognized registered institutions: {sorted(target_names)}")

    return registered_names_set

def filter_marksheet(marksheet_df, registered_names_set):
    registered_marksheet_df = marksheet_df[marksheet_df["Institution"].isin(registered_names_set)].copy()
    return registered_marksheet_df

In [ ]:
def distribute_slots(marksheet_df, total_slots, max_slots):
    # Replace NaN ratings with 0 and convert to integers
    marksheet_df['Rating'] = marksheet_df['Rating'].fillna(0).astype(int)
    
    institution_rating = dict(zip(marksheet_df['Institution'], marksheet_df['Rating']))
    allocated_slots = {institution: 1 for institution in institution_rating}
    total_allocated = len(allocated_slots)

    pq = heapdict()
    for institution, rating in institution_rating.items():
        pq[institution] = -rating / 2

    while total_allocated < total_slots and pq:
        institution, _ = pq.popitem()
        
        if allocated_slots[institution] < max_slots:
            allocated_slots[institution] += 1
            total_allocated += 1

            new_priority = -institution_rating[institution] / (allocated_slots[institution] + 1)
            pq[institution] = new_priority

    return pd.DataFrame({
        'Institution': allocated_slots.keys(),
        'Rating': institution_rating.values(),
        'Slots': allocated_slots.values()
    })

In [ ]:
registered_names_set = get_registered_institution_names(institutions, registered_institutions_file_path)
registered_marksheet_df = filter_marksheet(marksheet_df, registered_names_set)

In [ ]:
total_slots = options_map['total_slots']
max_slots = options_map['max_slots']
slots_df = distribute_slots(registered_marksheet_df, total_slots, max_slots)
display(slots_df)

# Generate Excel Sheet

In [ ]:
def round_floats_in_df(df, precision=6):
    rounded_df = df.copy()
    for col in rounded_df.select_dtypes(include='float'):
        rounded_df[col] = rounded_df[col].round(precision)
    return rounded_df

def col_letter(col_num):
    result = ''

    while col_num >= 0:
        result = chr(col_num % 26 + ord('A')) + result
        col_num = col_num // 26 - 1

    return result

In [ ]:
def write_header(worksheet, df):
    for col_num, col_name in enumerate(df.columns):
        worksheet.write(0, col_num, col_name)

def write_rows(slots_ws, df, general_col, reserved_col, total_col):
    for row in range(len(df)):
        excel_row = row + 2
        general_cell = f"{col_letter(general_col)}{excel_row}"
        reserved_cell = f"{col_letter(reserved_col)}{excel_row}"
        formula = f"={general_cell}+{reserved_cell}"
        slots_ws.write_formula(row + 1, total_col, formula)

In [ ]:
def apply_alternating_colors(workbook, worksheet, df, colors):
    header_format = workbook.add_format({'bg_color': colors[0], 'font_color': 'white'})
    worksheet.set_row(0, None, header_format)

    num_rows = len(df)
    num_cols = len(df.columns)
    col_range = f"A2:{col_letter(num_cols - 1)}{num_rows + 1}"

    worksheet.conditional_format(col_range, {
        'type': 'formula',
        'criteria': '=ISEVEN(ROW())',
        'format': workbook.add_format({'bg_color': colors[1]}),
    })

    worksheet.conditional_format(col_range, {
        'type': 'formula',
        'criteria': '=ISODD(ROW())',
        'format': workbook.add_format({'bg_color': colors[2]}),
    })

def autofit_columns(worksheet, df):
    for col_num, column in enumerate(df.columns):
        max_width = max(df[column].astype(str).map(len).max(), len(column)) + 2
        worksheet.set_column(col_num, col_num, max_width)

In [ ]:
def lock_sheet(worksheet):
    worksheet.protect()

def unlock_columns(worksheet, col_indices, start_row, end_row, df):
    unlocked_format = worksheet.book.add_format({'locked': False})
    for row in range(start_row, end_row + 1):
        for col in col_indices:
            value = df.iloc[row - 1, col]
            worksheet.write(row, col, value, unlocked_format)

def generateExcelSheet(marksheet_df, slots_df):
    slots_sheet_df = slots_df.copy()
    slots_sheet_df.rename(columns={'Slots': 'General Slots'}, inplace=True)
    slots_sheet_df['Reserved Slots'] = 0
    slots_sheet_df['Total Slots'] = ''
    slots_sheet_df['Explanation for Reserved Slots'] = ''

    slots_sheet_df = round_floats_in_df(slots_sheet_df)
    rounded_marksheet_df = round_floats_in_df(marksheet_df)

    os.makedirs('output', exist_ok=True)
    output_file = 'output/slots.xlsx'

    colors = ['#535FC1', '#C9DAF8', '#FFFFFF']

    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        workbook = writer.book

        slots_sheet_df.to_excel(writer, sheet_name='Slots', index=False, header=False, startrow=1)
        slots_ws = writer.sheets['Slots']
        slots_ws.book = workbook

        write_header(slots_ws, slots_sheet_df)

        general_col = slots_sheet_df.columns.get_loc('General Slots')
        reserved_col = slots_sheet_df.columns.get_loc('Reserved Slots')
        total_col = slots_sheet_df.columns.get_loc('Total Slots')
        explanation_col = slots_sheet_df.columns.get_loc('Explanation for Reserved Slots')

        write_rows(slots_ws, slots_sheet_df, general_col, reserved_col, total_col)
        
        apply_alternating_colors(workbook, slots_ws, slots_sheet_df, colors)
        autofit_columns(slots_ws, slots_sheet_df)

        lock_sheet(slots_ws)
        editable_cols = [reserved_col, explanation_col]
        unlock_columns(slots_ws, editable_cols, 1, len(slots_sheet_df), slots_sheet_df)

        rounded_marksheet_df.to_excel(writer, sheet_name='Marksheet', index=False, header=False, startrow=1)
        marksheet_ws = writer.sheets['Marksheet']
        marksheet_ws.book = workbook

        write_header(marksheet_ws, rounded_marksheet_df)
        apply_alternating_colors(workbook, marksheet_ws, rounded_marksheet_df, colors)
        autofit_columns(marksheet_ws, rounded_marksheet_df)

        lock_sheet(marksheet_ws)

In [ ]:
generateExcelSheet(registered_marksheet_df, slots_df)